In [11]:
import pandas
import pystan
import numpy
import scipy.stats

NeuronTopGeneGuides = pandas.read_csv("~/sgRNA/tiling/Neuron/NeuronTopGeneGuides.txt", sep='\t', header=0)
NeuronTopGeneGuides.head()

NeuronTopGeneGuides['gene'] = NeuronTopGeneGuides['gene'].astype('category')

mixture_model = """
data {
  int<lower=1> n_sgRNAs;
  int<lower=1> n_genes;
  real mu1;
  real<lower=0, upper=1> q0;
  real x[n_sgRNAs];
  int<lower=0, upper=n_genes> gene_ids[n_sgRNAs];
}
parameters {
  real mu_g[n_genes];
  real<lower=0, upper=1> q[n_genes];
  real<lower=0> sigma_g;
  real<lower=0> sigma1;
  real mu0;
  real<lower=0> sigma0;
}
model{
  mu_g ~ normal(mu1, sigma_g);
  mu0 ~ normal(0, 0.1);
  sigma0 ~ normal(0, 1);
  sigma_g ~ cauchy(0, 1);
  q ~ beta(q0*4/(1 - q0), 4);
  sigma1 ~ cauchy(0, 1);
  for (i in 1:n_sgRNAs){
    target += log_mix(q[gene_ids[i]], 
                      normal_lpdf(x[i] | mu_g[gene_ids[i]], sigma1), 
                      normal_lpdf(x[i] | mu0, sigma0)); 
  }
}
"""

sgRNAdata = {'n_sgRNAs' : NeuronTopGeneGuides.shape[0],
             'n_genes' : NeuronTopGeneGuides['gene'].nunique(),
             'mu1' : 1.7,
             'q0' : 0.2,
             'x' : NeuronTopGeneGuides['log2fc'],
             'gene_ids' : NeuronTopGeneGuides['gene'].cat.codes + 1 # stan starts counting at 1
            }

neuron_stan_fit = pystan.stan(model_code = mixture_model,
                             data=sgRNAdata, iter=2000, chains=4)
print(neuron_stan_fit)

mu_g_sample = pandas.DataFrame(neuron_stan_fit['mu_g'])
neuron_gene_means =  mu_g_sample.mean(axis=0)
q_sample = pandas.DataFrame(neuron_stan_fit['q'])
neuron_mixing = q_sample.mean(axis = 0)
sigma1_sample = pandas.DataFrame(neuron_stan_fit['sigma1'])
neuron_sigma1 = sigma1_sample.mean(axis = 0)
sigma0_sample = pandas.DataFrame(neuron_stan_fit['sigma0'])
neuron_sigma0 = sigma0_sample.mean(axis = 0)
mu0_sample = pandas.DataFrame(neuron_stan_fit['mu0'])
neuron_mu0 = mu0_sample.mean(axis = 0)

gene_ids = NeuronTopGeneGuides['gene'].cat.codes
neuron_sgRNA_probs = numpy.zeros(sgRNAdata['n_sgRNAs'])
gene_effects = numpy.zeros(sgRNAdata['n_sgRNAs'])
mixing = numpy.zeros(sgRNAdata['n_sgRNAs'])

for i in range(neuron_sgRNA_probs.shape[0]):
    gene = gene_ids[i]
    mu_g = neuron_gene_means[gene]
    gene_effects[i] = mu_g
    q = neuron_mixing[gene]
    mixing[i] = q
    pos_prob = q*scipy.stats.norm(mu_g, neuron_sigma1).pdf(NeuronTopGeneGuides['log2fc'][i])[0]
    neg_prob = (1 - q)*scipy.stats.norm(neuron_mu0, neuron_sigma0).pdf(NeuronTopGeneGuides['log2fc'][i])[0]
    neuron_sgRNA_probs[i] = pos_prob/(pos_prob + neg_prob)
    
NeuronTopGeneGuides['mixture_probs'] = neuron_sgRNA_probs  
NeuronTopGeneGuides['gene_effect'] = gene_effects
NeuronTopGeneGuides['mixing'] = mixing
NeuronTopGeneGuides.head()

NeuronTopGeneGuides.to_csv("NeuronTopGeneGuidesMixtureProbs.txt", sep='\t')




INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_83d9cc82176032cc96b5f1f9bcbe2f0d NOW.
/Users/Daley/miniconda2/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/8m/0f0r_lmn4p5bmrzymmp2d0qr0000gn/T/tmpFt1ZQj/stanfit4anon_model_83d9cc82176032cc96b5f1f9bcbe2f0d_2668197137305230409.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Inference for Stan model: anon_model_83d9cc82176032cc96b5f1f9bcbe2f0d.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu_g[1]    1.73  5.5e-3   0.33   1.03   1.59   1.71   1.85   2.48 3580.0    1.0
mu_g[2]    1.59  8.2e-3    0.3   0.83   1.48   1.64   1.75   2.12 1329.0    1.0
mu_g[3]    1.72  4.8e-3   0.32   1.08   1.58   1.71   1.84   2.47 4572.0    1.0
mu_g[4]    1.73  5.0e-3   0.32   1.08   1.59   1.71   1.86   2.47 4129.0    1.0
mu_g[5]     1.7  4.1e-3   0.28   1.11   1.57    1.7   1.83   2.31 4692.0    1.0
mu_g[6]    1.75  6.3e-3   0.33   1.12    1.6   1.71   1.86   2.55 2777.0    1.0
mu_g[7]    1.68  5.1e-3   0.32   0.96   1.55   1.68    1.8   2.36 3910.0    1.0
mu_g[8]     1.7  5.7e-3   0.32   1.04   1.56    1.7   1.82   2.43 3140.0    1.0
mu_g[9]    1.74  5.6e-3   0.31   1.15    1.6   1.71   1.86   2.48 3091.0    1.0
mu_g[10]   

In [12]:

SelfRenewalTopGeneGuides = pandas.read_csv("~/sgRNA/tiling/SelfRenewal/SelfRenewalTopGeneGuides.txt", sep='\t', header=0)
SelfRenewalTopGeneGuides.head()

SelfRenewalTopGeneGuides['gene'] = SelfRenewalTopGeneGuides['gene'].astype('category')


sgRNAdata = {'n_sgRNAs' : SelfRenewalTopGeneGuides.shape[0],
             'n_genes' : SelfRenewalTopGeneGuides['gene'].nunique(),
             'mu1' : 4.3,
             'q0' : 0.5,
             'x' : SelfRenewalTopGeneGuides['log2fc'],
             'gene_ids' : SelfRenewalTopGeneGuides['gene'].cat.codes + 1 # stan starts counting at 1
            }

selfrenewal_stan_fit = pystan.stan(model_code = mixture_model,
                             data=sgRNAdata, iter=2000, chains=4)
print(selfrenewal_stan_fit)

mu_g_sample = pandas.DataFrame(selfrenewal_stan_fit['mu_g'])
selfrenewal_gene_means =  mu_g_sample.mean(axis=0)
q_sample = pandas.DataFrame(selfrenewal_stan_fit['q'])
selfrenewal_mixing = q_sample.mean(axis = 0)
sigma1_sample = pandas.DataFrame(selfrenewal_stan_fit['sigma1'])
selfrenewal_sigma1 = sigma1_sample.mean(axis = 0)
sigma0_sample = pandas.DataFrame(selfrenewal_stan_fit['sigma0'])
selfrenewal_sigma0 = sigma0_sample.mean(axis = 0)
mu0_sample = pandas.DataFrame(selfrenewal_stan_fit['mu0'])
selfrenewal_mu0 = mu0_sample.mean(axis = 0)

gene_ids = SelfRenewalTopGeneGuides['gene'].cat.codes
selfrenewal_sgRNA_probs = numpy.zeros(sgRNAdata['n_sgRNAs'])
gene_effects = numpy.zeros(sgRNAdata['n_sgRNAs'])
mixing = numpy.zeros(sgRNAdata['n_sgRNAs'])
for i in range(selfrenewal_sgRNA_probs.shape[0]):
    gene = gene_ids[i]
    mu_g = selfrenewal_gene_means[gene]
    gene_effects[i] = mu_g
    q = selfrenewal_mixing[gene]
    mixing[i] = q
    pos_prob = q*scipy.stats.norm(mu_g, selfrenewal_sigma1).pdf(SelfRenewalTopGeneGuides['log2fc'][i])[0]
    neg_prob = (1 - q)*scipy.stats.norm(selfrenewal_mu0, selfrenewal_sigma0).pdf(SelfRenewalTopGeneGuides['log2fc'][i])[0]
    selfrenewal_sgRNA_probs[i] = pos_prob/(pos_prob + neg_prob)
    
SelfRenewalTopGeneGuides['mixture_probs'] = selfrenewal_sgRNA_probs   
SelfRenewalTopGeneGuides['gene_effect'] = gene_effects
SelfRenewalTopGeneGuides['mixing'] = mixing
SelfRenewalTopGeneGuides.head()

SelfRenewalTopGeneGuides.to_csv("SelfRenewalTopGeneGuidesMixtureProbs.txt", sep='\t')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_83d9cc82176032cc96b5f1f9bcbe2f0d NOW.
/Users/Daley/miniconda2/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/8m/0f0r_lmn4p5bmrzymmp2d0qr0000gn/T/tmpwnm52e/stanfit4anon_model_83d9cc82176032cc96b5f1f9bcbe2f0d_2667735655369205109.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


Inference for Stan model: anon_model_83d9cc82176032cc96b5f1f9bcbe2f0d.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu_g[1]    3.43  8.6e-3    0.6   2.27   3.02   3.41   3.82   4.63 4846.0    1.0
mu_g[2]    3.27    0.01   0.79   1.79   2.71   3.23   3.79   4.88 4299.0    1.0
mu_g[3]    2.52  6.1e-3   0.43   1.73   2.23   2.51   2.79   3.39 4865.0    1.0
mu_g[4]    2.97  4.4e-3   0.31   2.39   2.76   2.97   3.17   3.58 4898.0    1.0
mu_g[5]    4.22  8.9e-3   0.62   3.04   3.81   4.21   4.64   5.45 4810.0    1.0
mu_g[6]     2.9  6.4e-3   0.42   2.12    2.6   2.89   3.17   3.76 4294.0    1.0
mu_g[7]    3.52  4.4e-3    0.3   2.94   3.31   3.52   3.71   4.11 4588.0    1.0
mu_g[8]    3.16  5.9e-3   0.41   2.39   2.88   3.16   3.42   3.97 4687.0    1.0
mu_g[9]    4.51    0.01   0.86   2.89   3.89   4.49   5.07   6.29 5306.0    1.0
mu_g[10]   